In [3]:
result_by_role = {'default': {'hostname': 'beni.com', 'port': 3000}, u'hawkular': {'hostname': u'alon_gold.com', 'port': 1234}}
desired_by_role = {'default': {'hostname': 'erez_oshift01.com', 'port': 3000}}

# Original

In [8]:
updated = {role: {k: v for k, v in ep.items()
                  if k not in result_by_role[role] or v != result_by_role[role][k]}
           for role, ep in desired_by_role.items()
           if role in result_by_role and ep != result_by_role[role]}
added = {role: ep for role, ep in desired_by_role.items()
         if role not in result_by_role}
removed = {role: ep for role, ep in result_by_role.items()
           if role not in desired_by_role}
{"Updated": updated, "Added": added, "Removed": removed}


{'Added': {},
 'Removed': {u'hawkular': {'hostname': u'alon_gold.com', 'port': 1234}},
 'Updated': {'default': {'hostname': 'erez_oshift01.com'}}}

In [ ]:
updated = {role: {k: v for k, v in ep.items()
                  if k not in result_by_role[role] or v != result_by_role[role][k]}
           for role, ep in desired_by_role.items()
           if role in result_by_role and ep != result_by_role[role]}
added = {role: ep for role, ep in desired_by_role.items()
         if role not in result_by_role}
removed = {role: ep for role, ep in result_by_role.items()
           if role not in desired_by_role}
{"Updated": updated, "Added": added, "Removed": removed}


# Recursive, different {key: 'Updated'/...} results
Easier to generate recursively but less informative.

In [6]:
def dict_diff(old, new):
    diff = {}
    for k in set(old) - set(new):
        diff[k] = 'Removed'
    for k in set(new) - set(old):
        diff[k] = 'Added'
    for k in set(new) & set(old):
        if old[k] != new[k]:
            if isinstance(old[k], dict) and isinstance(new[k], dict):
                subdiff = dict_diff(old[k], new[k])
                if subdiff:
                    diff[k] = subdiff
            else:
                diff[k] = 'Updated'
    return diff

In [7]:
dict_diff(result_by_role, desired_by_role)

{'default': {'hostname': 'Updated'}, u'hawkular': 'Removed'}

In [10]:
# Another implementation, assuming None is not a valid value

def value_diff(old, new):
    if old == new:
        return None
    if isinstance(old, dict) and isinstance(new, dict):
        return dict_diff(old, new)
    if old is None:
        return 'Added'
    if new is None:
        return 'Removed'
    return 'Updated'

def dict_diff2(old, new):
    diff = {}
    for k in set(old) | set(new):
        subdiff = value_diff(old.get(k, None), new.get(k, None))
        if subdiff is not None:
            diff[k] = subdiff
    return diff

In [12]:
dict_diff2(result_by_role, desired_by_role)

{'default': {'hostname': 'Updated'}, u'hawkular': 'Removed'}